In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
import config, utils

# Create a Spark session
spark = utils.create_spark_session("playerStats", {
    'spark.executor.memory': '4g',
    'spark.executor.cores': '6',
})

fielding_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'fielding_data.csv')
bowling_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'bowling_data.csv')
batting_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'batting_data.csv')

fielding_data.show(5)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-15T11:55:25.377+0530] {utils.py:12} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/15 11:55:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/15 11:55:32 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


[2024-11-15T11:55:33.422+0530] {utils.py:30} INFO - Spark session created successfully.
[2024-11-15T11:55:33.424+0530] {utils.py:39} INFO - Loading data from fielding_data.csv.


[2024-11-15T11:55:46.114+0530] {utils.py:39} INFO - Loading data from bowling_data.csv.


[2024-11-15T11:55:46.894+0530] {utils.py:39} INFO - Loading data from batting_data.csv.
+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|    Player| Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|A Ahmadhel|Bulgaria|2019/20|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2020|             3|            3.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|2020/21|             4|            4.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2021|             6|            6.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2023|             9|            9.0|           0.0|    

In [2]:
# Check for nulls in the data
def check_nulls(df):
    nulls = 0
    for col in df.columns:
        nulls+= df.filter(df[col].isNull()).count()
    print("Total nulls in the dataframe: ", nulls)

check_nulls(batting_data)
check_nulls(bowling_data)
check_nulls(fielding_data)

Total nulls in the dataframe:  0
Total nulls in the dataframe:  0
Total nulls in the dataframe:  0


In [ ]:
print(batting_data.columns,batting_data.count())
print(bowling_data.columns,bowling_data.count())
print(fielding_data.columns,fielding_data.count())

['Player', 'Country', 'Season', 'Cum Mat Total', 'Cum Inns Total', 'Cum Runs Total', 'Cum Batting Ave', 'Cum SR'] 14140
['Player', 'Country', 'Season', 'Cumulative Mat', 'Cumulative Inns', 'Cumulative Overs', 'Cumulative Runs', 'Cumulative Wkts', 'Cumulative Econ'] 14140
['Player', 'Country', 'Season', 'Cumulative Mat', 'Cumulative Inns', 'Cumulative Dis', 'Cumulative Ct', 'Cumulative St', 'Cumulative D/I'] 14140


24/11/15 12:16:47 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.245.142: worker lost
24/11/15 13:17:44 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/11/15 13:17:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:218)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:923)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:262)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:169)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.s

In [4]:
playerdata = batting_data.join(bowling_data, on=['player_id','Player',"Country","Season"], how='inner').join(fielding_data, on=['player_id','Player',"Country","Season"], how='inner')\
    .drop('Cumulative Mat','Cumulative Inns')
playerdata = playerdata.dropDuplicates(['player_id', 'Player', 'Country', 'Season'])
playerdata.show(5)

24/11/15 01:51:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+---------------+---------+-------+-------------+--------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|player_id|         Player|  Country| Season|Cum Mat Total|Cum Runs Total|Cum SR|  Cumulative Overs|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------+---------------+---------+-------+-------------+--------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
| badef0dc|     MO Kitunda| Tanzania|2021/22|            0|             0|   0.0|               0.0|                    0.0|            0.0|            0.0|           0.0|          0.0|          0.0|           0.0|
| fbb657bb|    Babar Hayat|Hong Kong|2023/24|           45|          1067|115.74|               0.0|                    0.0|            0.0|

In [5]:
playerdata.sort(["Player","Country"]).show()

+---------+-----------+--------------+-------+-------------+--------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|player_id|     Player|       Country| Season|Cum Mat Total|Cum Runs Total|Cum SR|  Cumulative Overs|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------+-----------+--------------+-------+-------------+--------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
| 55a5cffb| A Ahmadhel|      Bulgaria|2020/21|            4|            24| 100.0|              12.0|                   97.0|            4.0|           8.38|           0.0|          0.0|          0.0|           0.0|
| 55a5cffb| A Ahmadhel|      Bulgaria|   2023|            9|            31| 63.42| 17.40000009536743|                  155.0|           

In [6]:
utils.spark_save_data(playerdata,config.PROCESSED_DATA_DIR, 'player_stats.csv')
spark.stop()

[2024-11-15T01:51:54.369+0530] {utils.py:57} INFO - Successfully wrote data to /usr/ravi/t20/data/2_processedData/player_stats.csv
